In [4]:
import numpy as np
import pandas as pd
import torch
from sklearn.datasets import load_diabetes
diabetes_data = load_diabetes()

print(diabetes_data.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [5]:
x = torch.from_numpy(np.array(diabetes_data.data[:, :-1], dtype=np.float32))
y= torch.from_numpy(np.array(diabetes_data.target, dtype=np.float32).reshape(-1,1))
print('shape of x is :', x.shape)
print('shape of y is :', y.shape)

shape of x is : torch.Size([442, 9])
shape of y is : torch.Size([442, 1])


In [6]:
y[1]

tensor([75.])

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class LinearRegressionModel(nn.Module):
    def __init__(self,i,o):
        super().__init__()
        self.linear = nn.Linear(i,o)
    def forward(self,x):
        return self.linear(x)

model = LinearRegressionModel(x.size(1), y.size(1))

In [8]:
lr = 0.01
np_epochs = 10000
opt = torch.optim.SGD(model.parameters(), lr = lr)

for epoch in range(np_epochs+1):
    y_pred = model(x)
    loss = F.mse_loss(y_pred,y)

    opt.zero_grad()
    loss.backward()
    opt.step()

In [9]:
print(loss)
for param in model.parameters():
    print(param)

tensor(3454.0461, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[  55.4240,  -28.2715,  268.1071,  190.3354,   42.6817,   16.1131,
         -153.7414,  142.4173,  240.4039]], requires_grad=True)
Parameter containing:
tensor([152.1331], requires_grad=True)


In [10]:
df = pd.DataFrame(torch.cat([y,y_pred],dim=1).detach().numpy(), columns=['y','y_pred'] )
import plotly.graph_objects as go
import plotly.offline as pyo

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['y'], mode='markers', name='y'))
fig.add_trace(go.Scatter(x=df.index, y=df['y_pred'], mode= 'markers', name='y_pred'))

fig.update_layout(yaxis_range=[-500,1000])
fig.show()